In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
def compute_beta_measures(G):

    # COMPUTE LAMBDAS (OUT-DEGREE)
    lambdas = dict(G.out_degree())

    # COMPUTE BETA-MEASURES
    beta_measures = {}
    for target_node in list(G.nodes()):
        beta = 0
        predecessors = list(G.predecessors(target_node))

        for node in predecessors:
            beta += G.get_edge_data(node, target_node)["weight"] / lambdas[node]

        beta_measures[target_node] = beta

    return beta_measures

In [19]:
# SET YEAR RANGE
first_year = 2010; last_year = 2019
years = [str(y) for y in range(first_year, last_year+1)]

# INITIALIZE DICTIONARY OF DATAFRAME FOR SURFERS
results_dict = {}

# LOOP OVER YEARS
for year in years:

    # LOAD NETWORK FOR YEAR
    year_net = nx.read_gml(f"../data/time_series_networks/overall_snapshot_{year}.gml")

    # GET IN-DEGREE AND BETA
    in_degrees = dict(year_net.in_degree(weight="weight"))
    betas = compute_beta_measures(year_net)
    deg_per_event = {}
    for surfer in in_degrees.keys():
        try:
            deg_per_event[surfer] = in_degrees[surfer] / year_net.nodes[surfer]["events"]
        except:
            deg_per_event[surfer] = 0

    # LOOP OVER NODES IN NETWORK
    for surfer in list(year_net.nodes()):

        # UPDATE DATAFRAME IF SURFER ALREADY BEING TRACKED
        if surfer in results_dict.keys():
            results_dict[surfer].loc[year] = [in_degrees[surfer], betas[surfer], deg_per_event[surfer]]

        # INITIALIZE DATAFRAME IF SURFER NOT BEING TRACKED
        else:
            results_dict[surfer] = pd.DataFrame(columns=["in-degree", "beta", "deg_per_event"])
            results_dict[surfer].loc[year] = [in_degrees[surfer], betas[surfer], deg_per_event[surfer]]

for key, df in results_dict.items():
    results_dict[key] = df.reset_index(names="year")

In [21]:
results_dict["Gabriel Medina"]

,year,in-degree,beta,deg_per_event
0,2011,75.0,4.881794,12.500000
1,2012,206.0,11.317442,12.875000
2,2013,256.0,9.997517,9.846154
3,2014,475.0,21.265102,12.837838
4,2015,657.0,25.280672,13.687500
5,2016,845.0,29.946074,14.322034
6,2017,1010.0,36.460331,14.428571
7,2018,1300.0,47.539103,16.049383
8,2019,1519.0,56.134790,16.510870
